# RNN을 이용한 텍스트 생성


    '경마장에 있는 말이 뛰고 있다'
    '그의 말이 법이다'
    '가는 말이 고와야 오는 말이 곱다'
<pre> 요고도 가능
</pre>

In [1]:
import numpy as np 
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [11]:
text = '''경마장에 있는 말이 뛰고 있다.
그의 말이 법이다.
가는 말이 고와야 오는 말이 곱다.'''

In [12]:
# print(text)

- 텍스트 전처리 영어처럼 스페이스를 기준

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [14]:
# 단어 집합 생성
t = Tokenizer()
t.fit_on_texts([text])

In [16]:
# 특징 
# 1. word_index 순서가 가나다 순서가 아니고, 많이 나온 단어 순
# 2. 0 index가 없다. 1번부터 시작한다.
t.word_index

{'가는': 8,
 '경마장에': 2,
 '고와야': 9,
 '곱다': 11,
 '그의': 6,
 '뛰고': 4,
 '말이': 1,
 '법이다': 7,
 '오는': 10,
 '있는': 3,
 '있다': 5}

In [18]:
# 단어 집합 크기 - keras Tokenizer의 시작 인덱스가 1이기 때문
# 케라스 토크나이저 쓸때는 늘 보캡사이즈에 1을 더해서 사용하자
vocab_size = len(t.word_index) + 1
vocab_size

12

In [20]:
t.texts_to_sequences(['그의 말이 법이다'])

[[6, 1, 7]]

In [22]:
# 학습에 사용될 샘플 시퀀스 
sequences = []

for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    print(encoded)

[2, 3, 1, 4, 5]
[6, 1, 7]
[8, 1, 9, 10, 1, 11]


In [24]:
sequences = []
for line in text.split('\n'):
    encoded=t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequences.append(encoded[:i+1])

sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [28]:
# 만든 리스트 중에 제일 긴놈이 뭐야? => max_len
max_len = max(len(seq) for seq in sequences)
max_len

6

### 패딩 두두등장.

 - Padding

        전체 샘플길이를 max_len에 맞추도록 0을 추가
        pre 옵션을 주면 앞쪽을 채움

In [29]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
sequences
# 아래는 RNN의 쉐입과 관련이 있어. 타입 스텝과, 인풋디멘션(피쳐개수)
# 맨 마지막 컬럼 3,1,4,5,~ 가 y값이 되는것. 그 앞은 x값이 될것 

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [30]:
X = sequences[:, :-1]
y = sequences[:, -1]

In [31]:
X.shape  # 시퀀스 개수 11개 타임스텝 5개 인풋디멘전은 단어의 개수가 보캡사이즈 만큼. 

(11, 5)

In [33]:
Y = to_categorical(y)
Y
# 순서좀 맞춰주면 안되나 컴퓨터야?

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

- 모델 정의
- 임베딩
- 심플알엔엔
- Dense